In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time

import pandas
pandas.set_option('display.max_rows', 1000)

from lsst.rsp import get_tap_service, retrieve_query

import lsst.daf.butler as dafButler

import lsst.geom
import lsst.afw.display as afwDisplay

# Butler

In [ ]:
# collection = '2.2i/runs/test-med-1/v23_0_0_rc2/PREOPS-863'
# collection = '2.2i/runs/DP0.2/v23_0_2/PREOPS-905/step_all'

In [3]:
butler = dafButler.Butler('dp02')
registry = butler.registry

In [5]:
# for c in sorted(registry.queryCollections()):
#     print(c)

## Look for truth data

In [22]:
config = 'dp02'

In [23]:
collection = '2.2i/truth_summary'

In [24]:
butler = dafButler.Butler(config, collections=collection)
registry = butler.registry

In [25]:
for c in sorted(registry.queryCollections()):
    temp = str(c)
    if temp.find('truth') > -1:
        print(temp)

2.2i/truth_summary
2.2i/truth_summary/ci_imsim


In [26]:
for x in sorted(registry.queryDatasetTypes()):
    temp = str(x)
    if temp.find('truth') > -1:
        print(temp)

DatasetType('diff_matched_truth_summary_objectTable_tract', {skymap, tract}, DataFrame)
DatasetType('match_ref_truth_summary_objectTable_tract', {skymap, tract}, DataFrame)
DatasetType('match_target_truth_summary_objectTable_tract', {skymap, tract}, DataFrame)
DatasetType('matched_truth_summary_objectTable_tract', {skymap, tract}, DataFrame)
DatasetType('truth_summary', {skymap, tract}, DataFrame)


In [27]:
for c in sorted(registry.queryCollections(collection, flattenChains=True)):
    print(c, registry.getCollectionType(c))

2.2i/truth_summary CollectionType.RUN


In [28]:
### For example, grab truth_summary for a tract
dataId = {'tract': 3829}
data = butler.get('truth_summary', dataId)

In [29]:
data.columns

Index(['id', 'host_galaxy', 'ra', 'dec', 'redshift', 'is_variable',
       'is_pointsource', 'flux_u', 'flux_g', 'flux_r', 'flux_i', 'flux_z',
       'flux_y', 'flux_u_noMW', 'flux_g_noMW', 'flux_r_noMW', 'flux_i_noMW',
       'flux_z_noMW', 'flux_y_noMW', 'tract', 'patch', 'truth_type',
       'cosmodc2_hp', 'cosmodc2_id', 'mag_r', 'match_objectId', 'match_sep',
       'is_good_match', 'is_nearest_neighbor', 'is_unique_truth_entry'],
      dtype='object')

<br><br>

## DiaObjects

DiaObjects via the Butler.

In [ ]:
config = 'dp02'
collection = '2.2i/runs/DP0.2/v23_0_2/PREOPS-905/step_all'
butler = dafButler.Butler(config, collections=collection)

In [ ]:
import lsst.sphgeom
pixelization = lsst.sphgeom.HtmPixelization(11)

In [ ]:
htm_id = pixelization.index(
    lsst.sphgeom.UnitVector3d(
        lsst.sphgeom.LonLat.fromDegrees(57.5, -36.5)
    )
)

# Obtain and print the scale to provide a sense of the size of the sky pixelization being used
scale = pixelization.triangle(htm_id).getBoundingCircle().getOpeningAngle().asDegrees()*3600
print(f'HTM ID={htm_id} at level={pixelization.getLevel()} is a ~{scale:0.2}" triangle.')

In [ ]:
datasetRefs = registry.queryDatasets("diaObjectTable_tract", htm20=htm_id)

In [ ]:
for i, ref in enumerate(datasetRefs):
    print(i, ref)
    if i > 6:
        break

In [ ]:
dataId = {'tract': 3829}
my_diaObjects = butler.get('diaObjectTable_tract', dataId)
my_diaSources = butler.get('diaSourceTable_tract', dataId)

In [ ]:
print(len(my_diaObjects))
print(len(my_diaSources))

In [ ]:
# my_diaObjects

In [ ]:
diaO_id  = my_diaObjects.index.to_numpy()
diaO_ra  = my_diaObjects.ra.to_numpy()
diaO_dec = my_diaObjects.decl.to_numpy()

diaO_gPSMagMin = -2.5 * np.log10(my_diaObjects.gPSFluxMax.to_numpy()/1.0e32) - 48.60
diaO_rPSMagMin = -2.5 * np.log10(my_diaObjects.rPSFluxMax.to_numpy()/1.0e32) - 48.60
diaO_iPSMagMin = -2.5 * np.log10(my_diaObjects.iPSFluxMax.to_numpy()/1.0e32) - 48.60

diaO_nDiaSources = my_diaObjects.nDiaSources.to_numpy()

In [ ]:
# plt.hist( diaO_nDiaSources, bins=20, log=True )
# plt.show()

In [ ]:
# plt.hist( diaO_gPSMagMin, bins=20, log=True )
# plt.show()

In [ ]:
# plt.plot( diaO_ra, diaO_dec, 'o', ms=2, alpha=0.05 )
# plt.show()

In [ ]:
# my_diaSources

In [ ]:
diaS_diaObjectId = my_diaSources.diaObjectId.to_numpy()

diaS_filterName  = my_diaSources.filterName.to_numpy()
diaS_midPointTai = my_diaSources.midPointTai.to_numpy()
diaS_psFlux      = my_diaSources.psFlux.to_numpy()
diaS_psFluxErr   = my_diaSources.psFluxErr.to_numpy()
diaS_psMag       = -2.5 * np.log10(diaS_psFlux/1.0e32) - 48.60

upperflux        = diaS_psFlux + diaS_psFluxErr
lowermag         = -2.5 * np.log10(upperflux/1.0e32) - 48.60
diaS_psMagErr    = diaS_psMag - lowermag
del upperflux, lowermag

Making durations takes a while so probably you only want to do this for the potential SNIa-like ones in reality.

In [ ]:
# diaO_duration = np.zeros( len(diaO_id), dtype='float' )

# for i in range(len(diaO_id)):
#     sx = np.where( diaS_diaObjectId == diaO_id[i] )[0]
#     if len(sx) > 1:
#         diaO_duration[i] = np.max(diaS_midPointTai[sx]) - np.min(diaS_midPointTai[sx])

In [ ]:
# plt.hist( diaO_duration, bins=20, log=True )
# plt.show()

In [ ]:
# tx = np.where( (diaO_nDiaSources > 15) & \
#                (diaO_rPSMagMin > 18.0) & \
#                (diaO_rPSMagMin < 22.5) & \
#                (diaO_duration > 10.0)  & \
#                (diaO_duration < 200.0) )[0]
# print(len(tx))

In [ ]:
# plt.hist( diaO_nDiaSources[tx], bins=20 )
# plt.show()

In [ ]:
# filter_names = ['u', 'g', 'r', 'i', 'z', 'y']
# filter_color = ['darkviolet', 'darkgreen', 'red', 'darkorange', 'brown', 'black']
# filter_symbol = ['o', '^', 'v', 's', '*', 'p']

In [ ]:
# fig, ax = plt.subplots( 3, 5, figsize=(20,10), sharey=False, sharex=False)

# i = 0
# j = 0

# for t in tx:
#     sx = np.where( diaS_diaObjectId == diaO_id[t] )[0]
#     for f, filt in enumerate(filter_names):
#         fx = np.where(diaS_filterName[sx] == filt)[0]
#         ax[i,j].plot(diaS_midPointTai[sx[fx]], diaS_psMag[sx[fx]],
#                        filter_symbol[f], ms=15, mew=0, alpha=0.5, color=filter_color[f])
#         del fx
#     ax[i,j].set_ylim([np.nanmax(diaS_psMag[sx])+0.2, np.nanmin(diaS_psMag[sx])-0.2])
#     j += 1
#     if j == 5:
#         j = 0
#         i += 1
    
# plt.show()